In [1]:
import torch
import torch.nn as nn  
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, random_split
import torchvision
import torchvision.transforms as transforms
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
import os
from sklearn.metrics import classification_report
import ResNet as resnet
import DenseNet as densenet
import EfficientNet as efficientnet
import MobileNetV2 as mobilenet
import ViT as vit
import solver.solver as solver
import solver.solver_v2 as solver_v2
import solver.solver_v3 as solver_v3
from opacus import PrivacyEngine
from opacus.validators import ModuleValidator

/home/hyunjun/anaconda3/envs/test/lib/python3.7/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /home/hyunjun/anaconda3/envs/test/lib/python3.7/site-packages/torchvision/image.so: undefined symbol: _ZNK3c1010TensorImpl36is_contiguous_nondefault_policy_implENS_12MemoryFormatE
  warn(f"Failed to load image Python extension: {e}")


In [2]:
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "3"

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

n_epochs = 30
batch_size = 512
learning_rate = 1e-4
diff_drop = "v3"
opacus_baseline = False

model = resnet.resnet18(num_classes=8, diff_drop=diff_drop)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [3]:
dataset_path = "/media/data2/data/CelebAMask-HQ/CelebA-HQ-img/"
annotation_path = "/media/data2/data/CelebAMask-HQ/CelebAMask-HQ-attribute-anno.txt"

In [4]:
class CelebA(Dataset):
    def __init__(self, root, annotation_path, transform=None):
        self.transform = transform
        self.root_dir = root
        self.classes = [
            "Female, Not smiling, Young", "Female, Not smiling, Old", 
            "Female, Smiling, Young", "Female, Smiling, Old", 
            "Male, Not smiling, Young", "Male, Not smiling, Old", 
            "Male, Smiling, Young", "Male, Smiling, Old",
        ]
        
        self.image_paths = []
        self.image_labels = []

        anno_file = open(annotation_path, "r")
        num = int(anno_file.readline())
    
        line = anno_file.readline()
        attributes = list(line[:-1].split(' '))
        
        sex_index = attributes.index('Male') + 2
        smile_index = attributes.index('Smiling') + 2
        young_index = attributes.index('Young') + 2
        for _ in range(num):
            line = anno_file.readline()
            record = list(line[:-1].split(' '))
            self.image_paths.append(record[0])
            if record[sex_index] == "1":
                if record[smile_index] == "1":
                    if record[young_index] == "1":
                        self.image_labels.append(6)
                    else:
                        self.image_labels.append(7)
                else:
                    if record[young_index] == "1":
                        self.image_labels.append(4)
                    else:
                        self.image_labels.append(5)
            else:
                if record[smile_index] == "1":
                    if record[young_index] == "1":
                        self.image_labels.append(2)
                    else:
                        self.image_labels.append(3)
                else:
                    if record[young_index] == "1":
                        self.image_labels.append(0)
                    else:
                        self.image_labels.append(1)
        anno_file.close()
        
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, index):
        image_file_path = os.path.join(self.root_dir, self.image_paths[index])
        image = Image.open(image_file_path)

        label = self.image_labels[index]
        
        if self.transform is not None:
            image = self.transform(image)
        
        return image, label

In [5]:
trans = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

celebAset = CelebA(root=dataset_path, annotation_path=annotation_path, transform=trans)
train_size = int(0.7 * len(celebAset))
trainset, testset = random_split(celebAset, [train_size, len(celebAset) - train_size])

train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, drop_last=False, )
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=True, drop_last=False, )

classes = celebAset.classes
print(classes)
print(len(trainset), len(testset), len(celebAset))

privacy_engine = PrivacyEngine()
if opacus_baseline:
    if not ModuleValidator.is_valid(model):
        model = ModuleValidator.fix(model)
        optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    
    model, optimizer, train_loader = privacy_engine.make_private(
        module=model,
        optimizer=optimizer,
        data_loader=train_loader,
        noise_multiplier=1.1,
        max_grad_norm=100.0,
    )
        
model = model.to(device)

['Female, Not smiling, Young', 'Female, Not smiling, Old', 'Female, Smiling, Young', 'Female, Smiling, Old', 'Male, Not smiling, Young', 'Male, Not smiling, Old', 'Male, Smiling, Young', 'Male, Smiling, Old']
21000 9000 30000


/home/hyunjun/anaconda3/envs/test/lib/python3.7/site-packages/opacus/privacy_engine.py:143: UserWarning: Secure RNG turned off. This is perfectly fine for experimentation as it allows for much faster training performance, but remember to turn it on and retrain one last time before production with ``secure_mode`` turned on.
  "Secure RNG turned off. This is perfectly fine for experimentation as it allows "


In [6]:
def train_loop(dataloader, model, criterion, optimizer, epoch):
    size = len(dataloader.dataset)
    total_correct = 0
    total_loss = 0
    
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        model.train()
        correct = 0
        
        pred = None
        p = 0.0
        if diff_drop == "v3":
            pred, p = model(X.to(device), epoch)
        else:
            pred, p = model(X.to(device))
        
        loss = criterion(pred, y.to(device))
        correct += (torch.argmax(pred, dim=1) == y.to(device)).type(torch.float).sum().item()
        total_correct += correct
        
        model.train()
        total_loss += loss.item()
        
        optimizer.zero_grad()
        loss.backward()
        
        if diff_drop == "v1":
            solver.PseudoPruning(model.fc, p)
        elif diff_drop == "v2":
            solver_v2.PseudoPruning(model.fc, p)
        elif diff_drop == "v3":
            solver_v3.PseudoPruning(model.fc, p)
        
        optimizer.step()
        
        if (batch % 50 == 0):
            print(f"Batch {batch:>5d}: Loss per batch is {loss.item():>7.6f} and Accuracy is {correct / batch_size:>7.6f}")
    print(f"For Iteration: Loss is {(total_loss / (size // batch_size)):>7.6f} and Accuracy is {total_correct / size:>7.6f}")

In [7]:
def test_loop(dataloader, model, criterion):
    test_loss = 0
    correct = 0

    model.eval()
    
    y_true = []
    y_pred = []
    
    with torch.no_grad():
        for X, y in dataloader:
            pred, _ = model(X.to(device))
            test_loss += criterion(pred, y.to(device)).item()
            
            pred = torch.argmax(pred, dim=1)
            correct += (pred == y.to(device)).type(torch.float).sum().item()
            
            pred = pred.cpu().numpy()
            y = y.cpu().numpy()
            y_true.extend(y)
            y_pred.extend(pred)
    
    print(classification_report(y_true=y_true, y_pred=y_pred, target_names=classes))
    
    return correct / len(dataloader.dataset)

In [8]:
max = 0.0
for epoch in range(n_epochs):
    print(f"Training: Epoch {epoch + 1} ------------------")
    train_loop(train_loader, model, criterion, optimizer, epoch + 1)
    print(f"----Test: Epoch {epoch + 1} ------------------")
    temp = test_loop(test_loader, model, criterion)
    
    # if temp > max:
    #     max = temp
    #     torch.save(model, './celebAHQ_split_effib0_v3.pt')

Training: Epoch 1 ------------------
Batch     0: Loss per batch is 2.065235 and Accuracy is 0.166016
For Iteration: Loss is 1.706284 and Accuracy is 0.374667
----Test: Epoch 1 ------------------


/home/hyunjun/anaconda3/envs/test/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hyunjun/anaconda3/envs/test/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hyunjun/anaconda3/envs/test/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

                            precision    recall  f1-score   support

Female, Not smiling, Young       0.29      1.00      0.45      2585
  Female, Not smiling, Old       0.00      0.00      0.00       198
    Female, Smiling, Young       0.00      0.00      0.00      2447
      Female, Smiling, Old       0.00      0.00      0.00       459
  Male, Not smiling, Young       0.31      0.01      0.02      1226
    Male, Not smiling, Old       0.00      0.00      0.00       749
      Male, Smiling, Young       0.00      0.00      0.00       747
        Male, Smiling, Old       0.00      0.00      0.00       589

                  accuracy                           0.29      9000
                 macro avg       0.08      0.13      0.06      9000
              weighted avg       0.13      0.29      0.13      9000

Training: Epoch 2 ------------------
Batch     0: Loss per batch is 1.406824 and Accuracy is 0.498047
For Iteration: Loss is 1.330286 and Accuracy is 0.527238
----Test: Epoch 2 ----

/home/hyunjun/anaconda3/envs/test/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hyunjun/anaconda3/envs/test/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hyunjun/anaconda3/envs/test/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

                            precision    recall  f1-score   support

Female, Not smiling, Young       0.64      0.45      0.53      2585
  Female, Not smiling, Old       0.00      0.00      0.00       198
    Female, Smiling, Young       0.51      0.86      0.64      2447
      Female, Smiling, Old       0.57      0.01      0.02       459
  Male, Not smiling, Young       0.44      0.56      0.50      1226
    Male, Not smiling, Old       0.42      0.19      0.26       749
      Male, Smiling, Young       0.45      0.19      0.27       747
        Male, Smiling, Old       0.31      0.46      0.37       589

                  accuracy                           0.50      9000
                 macro avg       0.42      0.34      0.32      9000
              weighted avg       0.51      0.50      0.46      9000

Training: Epoch 3 ------------------
Batch     0: Loss per batch is 1.344609 and Accuracy is 0.486328
For Iteration: Loss is 1.132932 and Accuracy is 0.608524
----Test: Epoch 3 ----

...

/home/hyunjun/anaconda3/envs/test/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hyunjun/anaconda3/envs/test/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hyunjun/anaconda3/envs/test/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

                            precision    recall  f1-score   support

Female, Not smiling, Young       0.84      0.93      0.88      2585
  Female, Not smiling, Old       0.00      0.00      0.00       198
    Female, Smiling, Young       0.81      0.93      0.87      2447
      Female, Smiling, Old       0.76      0.39      0.52       459
  Male, Not smiling, Young       0.74      0.73      0.74      1226
    Male, Not smiling, Old       0.57      0.61      0.59       749
      Male, Smiling, Young       0.82      0.60      0.69       747
        Male, Smiling, Old       0.70      0.65      0.67       589

                  accuracy                           0.78      9000
                 macro avg       0.66      0.60      0.62      9000
              weighted avg       0.76      0.78      0.77      9000

Training: Epoch 29 ------------------
Batch     0: Loss per batch is 1.867885 and Accuracy is 0.273438
For Iteration: Loss is 1.863139 and Accuracy is 0.246333
----Test: Epoch 29 --

/home/hyunjun/anaconda3/envs/test/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hyunjun/anaconda3/envs/test/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hyunjun/anaconda3/envs/test/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

                            precision    recall  f1-score   support

Female, Not smiling, Young       0.61      0.99      0.75      2585
  Female, Not smiling, Old       0.00      0.00      0.00       198
    Female, Smiling, Young       0.81      0.77      0.79      2447
      Female, Smiling, Old       0.75      0.21      0.33       459
  Male, Not smiling, Young       0.55      0.56      0.56      1226
    Male, Not smiling, Old       0.54      0.29      0.38       749
      Male, Smiling, Young       0.80      0.45      0.58       747
        Male, Smiling, Old       0.83      0.40      0.54       589

                  accuracy                           0.67      9000
                 macro avg       0.61      0.46      0.49      9000
              weighted avg       0.68      0.67      0.64      9000

Training: Epoch 30 ------------------
Batch     0: Loss per batch is 1.162651 and Accuracy is 0.652344
For Iteration: Loss is 1.244878 and Accuracy is 0.560333
----Test: Epoch 30 --

/home/hyunjun/anaconda3/envs/test/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hyunjun/anaconda3/envs/test/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/hyunjun/anaconda3/envs/test/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

In [9]:
torch.save(model, './celebA_split_resnet18_v3_batch512.pt')